In [1]:
import os
import subprocess
import re

FOLDER_TOPIC = 'hotel'
MODELS = ["llama-3.1-70b-versatile", "llama-3.1-70b-versatile"]
PATH_WEBSITE = "/Users/arnobeauger/Desktop/MACHINE LEARNING/DeafDialogues/website_files"
PATH_HOTEL_FILE = os.path.join(PATH_WEBSITE, "hotel.html")

In [2]:
def read_file(filename):
    """Read the content of a file."""
    with open(filename, 'r', encoding='utf-8') as file:
        return file.read()

def write_file(filename, content):
    """Write content to a file."""
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)

def format_conversation(conversation_text):
    """Format the conversation text with HTML tags."""
    lines = conversation_text.split('\n')
    formatted_lines = []
    for line in lines:
        # Use regex to match the entity name pattern including surnames in parentheses
        match = re.match(r'^(.*?\(.*?\)):', line)
        if match:
            entity_name = match.group(1)
            message = line[len(entity_name)+1:].strip()
            formatted_line = f'<p><strong style="font-size: 1.05em;">{entity_name}:</strong> {message}</p>'
        else:
            formatted_line = f'<p>{line}</p>'
        formatted_lines.append(formatted_line)
    return '\n'.join(formatted_lines)

In [3]:
def extract_surnames(conversation_text):
    """Extract surnames (text in parentheses) from the conversation."""
    surnames = re.findall(r'\((.*?)\)', conversation_text)
    return list(dict.fromkeys(surnames))  # Remove duplicates while preserving order

In [4]:
def generate_conversation_html(template_html, conversation_text, conversation_number, surnames, output_filename):
    """Generate a new HTML file with updated conversation content."""
    start_tag = '<div class="conversation-content">'
    end_tag = '</div>'
    
    start_index = template_html.index(start_tag) + len(start_tag)
    end_index = template_html.index(end_tag, start_index)
    
    formatted_conversation = format_conversation(conversation_text)
    
    new_html = (
        template_html[:start_index] +
        f"\n{formatted_conversation}\n" +
        template_html[end_index:]
    )
    
    new_html = new_html.replace(
        '<title>Conversation 1 - Hotel of Consciousness</title>',
        f'<title>Conversation {conversation_number} - Hotel of Consciousness</title>' # ({" and ".join(surnames)})
    )
    
    write_file(output_filename, new_html)

In [5]:
def update_hotel_page(hotel_html, new_conversation_number, surnames):
    """Update the Hotel of Consciousness page with a new conversation link."""
    insert_point = hotel_html.index('</ul>')
    new_link = f'            <li><a href="conversation_{new_conversation_number}_{"_".join(MODELS)}.html">conversation_{new_conversation_number} ({" and ".join(MODELS)})</a></li>\n' #_{"_".join(surnames)}    c
    
    updated_html = (
        hotel_html[:insert_point] +
        new_link +
        hotel_html[insert_point:]
    )
    
    return updated_html

In [6]:
def main():
    # Run dialogues.py to generate new txt file
    # subprocess.run(["python", "dialogues.py"])    
    
    # Determine the new conversation number
    existing_conversations = [f for f in os.listdir(FOLDER_TOPIC) if f.startswith("conversation_") and f.endswith(".html")]
    new_conversation_number = len(existing_conversations) + 1
    
    # Paths
    template_file = f"{FOLDER_TOPIC}/template_conv.html"
    conversation_file = f"{FOLDER_TOPIC}/conversation_{new_conversation_number}.txt"
    # hotel_file = f"{FOLDER_TOPIC}/hotel.html"
    hotel_file = PATH_HOTEL_FILE
    
    
    # Read files
    template_html = read_file(template_file)
    conversation_text = read_file(conversation_file)
    hotel_html = read_file(hotel_file)
    
    # Extract surnames
    surnames = extract_surnames(conversation_text)
    
    # Generate output filename
    output_file = os.path.join(PATH_WEBSITE, f"conversation_{new_conversation_number}_{'_'.join(MODELS)}.html")
    
    # Generate new conversation HTML
    generate_conversation_html(template_html, conversation_text, new_conversation_number, MODELS, output_file)
    
    # Update Hotel of Consciousness page
    updated_hotel_html = update_hotel_page(hotel_html, new_conversation_number, MODELS)
    write_file(hotel_file, updated_hotel_html)
    
    print(f"Generated {output_file} and updated {hotel_file} successfully!")

In [7]:
if __name__ == "__main__":
    main()

Generated /Users/arnobeauger/Desktop/MACHINE LEARNING/DeafDialogues/website_files/conversation_1_llama-3.1-70b-versatile_llama-3.1-70b-versatile.html and updated /Users/arnobeauger/Desktop/MACHINE LEARNING/DeafDialogues/website_files/hotel.html successfully!
